## Connettore

In [2]:
from dotenv import load_dotenv
import os
load_dotenv()

admin


In [3]:
from src.connettori.db_manager import DatabaseManager

db = DatabaseManager(
    host="localhost",
    port="5432",
    database="postgres",
    user="postgres",
    password=os.getenv("DB_PWD")
)

In [4]:
if db.connect():
    query = "SELECT * FROM video_games.game LIMIT 5;"
    df = db.execute_query(query)
    
    if df is not None:
        print(df.head())
    
    db.close()

   id  genre_id                                game_name
0   1        11                              '98 Koshien
1   2         8               .hack//G.U. Vol.1//Rebirth
2   3         8             .hack//G.U. Vol.2//Reminisce
3   4         8  .hack//G.U. Vol.2//Reminisce (jp sales)
4   5         8            .hack//G.U. Vol.3//Redemption


## OpenAI

In [5]:
from src.openAI.openai_handler import OpenAIManager

openai_manager = OpenAIManager(
    api_key=os.getenv("OPENAI_API_KEY"),
    embedding_model="text-embedding-3-large",
    chat_model="gpt-4o"
)

In [6]:
text = "Questo è un esempio di testo"
embedding = openai_manager.get_embedding(text)

In [8]:
prompt = "Qual è la capitale dell'Italia?"
response = openai_manager.get_completion(prompt)
response

"La capitale dell'Italia è Roma."

In [ ]:
texts = ["testo 1", "testo 2", "testo 3"]
embeddings = openai_manager.get_embeddings_batch(texts)

In [9]:
for chunk in openai_manager.get_chat_stream("Raccontami una storia"):
    print(chunk, end='', flush=True)

C'era una volta un piccolo villaggio nascosto tra le colline verdeggianti di un'antica valle. Questo villaggio era unico nel suo genere, poiché era abitato da persone che possedevano un dono speciale: ognuno di loro aveva la capacità di comunicare con la natura. Gli abitanti del villaggio parlavano con gli alberi, ascoltavano le storie dei venti e comprendevano i segreti dei fiumi.

Tra tutti gli abitanti, c'era una giovane ragazza di nome Lila. Lila aveva una connessione particolarmente forte con la natura, tanto che spesso trascorreva le sue giornate nel bosco, conversando con gli animali e giocando con le foglie che danzavano al ritmo della brezza.

Un giorno, mentre esplorava una parte del bosco che non aveva mai visitato prima, Lila trovò un'antica quercia con una cavità nel tronco, all'interno della quale brillava una luce dorata. Avvicinandosi, Lila sentì una voce gentile che le parlava: era lo spirito del bosco, che le raccontò di un grande pericolo che minacciava l'equilibrio 

## OLLAMA

In [1]:
from src.ollama_.ollama_manager import OllamaManager

ollama_manager = OllamaManager(
    base_url="http://localhost:11434",
    model="llama3.2"
)

In [2]:
# Ottenere una risposta semplice
response = ollama_manager.get_completion(
    prompt="Qual è la capitale dell'Italia?",
    temperature=0.2
)
print(response)

La capitale dell'Italia è Roma.


In [3]:
# Utilizzare lo streaming
for chunk in ollama_manager.get_completion(
    prompt="Raccontami una storia",
    stream=True
):
    print(chunk, end='', flush=True)

Era una notte buia e fredda, e la luna era nascosta dietro le nuvole. Una giovane donna di nome Sofia si trovava a piedi in un bosco silenzioso, circondata da alberi alti e oscuri che sembravano estendere le loro braccia come giganti.

Sofia era una ragazza con un cuore pieno di speranza e un'anima piena di curiosità. Era sempre stata attratta dal mistero del bosco, che sembrava nascondere segreti e storie ancora non scoperte.

Mentre camminava, la sua mente cominciò a vagare tra i pensieri più vari. Pensava alla sua vita, alle sue amicizie, ai suoi sogni. Ma il suo spirito era sempre pronto ad esplorare nuovi orizzonti e a scoprire nuove verità.

Improvvisamente, Sofia sentì un rumore strano provenire da una direzione lontana. Si fermò e ascoltò attentamente, ma non riusciva a capire cosa fosse. Il rumore sembrava venire da una piccola cabina di legno che si trovava ai margini del bosco.

Sofia si avvicinò alla cabina con cautela, sentendo il cuore battere forte nel petto. La cabina e

In [4]:
# Vedere i modelli disponibili
models = ollama_manager.list_models()
print("Modelli disponibili:", models)

Modelli disponibili: [{'name': 'llama3.2:latest', 'model': 'llama3.2:latest', 'modified_at': '2024-10-26T14:04:45.5231945+02:00', 'size': 2019393189, 'digest': 'a80c4f17acd55265feec403c7aef86be0c25983ab279d83f3bcd3abbcb5b8b72', 'details': {'parent_model': '', 'format': 'gguf', 'family': 'llama', 'families': ['llama'], 'parameter_size': '3.2B', 'quantization_level': 'Q4_K_M'}}]


In [ ]:
# Scaricare un nuovo modello
success = ollama_manager.pull_model("mistral")
if success:
    print("Modello scaricato con successo")

# Cambiare modello
ollama_manager.set_model("mistral")

# Ottenere informazioni sul modello
model_info = ollama_manager.get_model_info()
print("Informazioni sul modello:", model_info)

## Flow di tutto

In [1]:
from src.dbcontext.schema_context_manager import SchemaContextManager
from src.connettori.db_manager import DatabaseManager
from src.prompt.prompt_manager import PromptManager
from src.ollama_.ollama_manager import OllamaManager

ollama_manager = OllamaManager(
    base_url="http://localhost:11434",
    model="llama3.1"
)

db = DatabaseManager(
    host="localhost",
    port="5432",
    database="postgres",
    user="postgres",
    password="admin"
)

db.connect()

schema_manager = SchemaContextManager(db.engine, schema ="video_games")
prompt_manager = PromptManager(schema_manager, db)

Cercando tabelle nello schema: video_games
Tabelle trovate: ['genre', 'publisher', 'platform', 'region', 'game', 'game_publisher', 'game_platform', 'region_sales']

Processando tabella: genre
Tabella genre processata con successo

Processando tabella: publisher
Tabella publisher processata con successo

Processando tabella: platform
Tabella platform processata con successo

Processando tabella: region
Tabella region processata con successo

Processando tabella: game
Tabella game processata con successo

Processando tabella: game_publisher
Tabella game_publisher processata con successo

Processando tabella: game_platform
Tabella game_platform processata con successo

Processando tabella: region_sales
Tabella region_sales processata con successo


In [9]:
# Utilizzo
question = "mostrami dati a tuo piacere dalla tabella 'game', mettendo una limit 5 e insrendo la query SQL tra ```sql ``` come richiesto"
prompt = prompt_manager.generate_prompt(question)
print(prompt)

You are an SQL expert who helps convert natural language queries into SQL queries.
        Your task is:
        1. Generate a valid SQL query that answers the user's question
        2. Provide a brief explanation of the results
        
        Response format:
        ```sql
        -- Your SQL query here
        ```
        Always insert schema name before the tables in sql query. The schema name is "video_games"
        Explanation: [Brief explanation of the query and expected results].
        

Schema del Database:

Tabella: genre (12 righe)
Colonne:
- id (INTEGER) NOT NULL
- genre_name (VARCHAR(50)) NULL
Chiavi Primarie: id

Tabella: publisher (577 righe)
Colonne:
- id (INTEGER) NOT NULL
- publisher_name (VARCHAR(100)) NULL
Chiavi Primarie: id

Tabella: platform (31 righe)
Colonne:
- id (INTEGER) NOT NULL
- platform_name (VARCHAR(50)) NULL
Chiavi Primarie: id

Tabella: region (4 righe)
Colonne:
- id (INTEGER) NOT NULL
- region_name (VARCHAR(50)) NULL
Chiavi Primarie: id

Tabell

In [10]:
# Ottieni la risposta dal modello (usando OpenAI o Ollama)
llm_response = ollama_manager.get_completion(prompt)
results = prompt_manager.process_query(llm_response)

if results["success"]:
    print("\nQuery SQL generata:")
    print(results["query"])
    print("\nSpiegazione:")
    print(results["explanation"])
    print("\nAnteprima risultati:")
    print(results["preview"])
else:
    print("Errore:", results["error"])


Query SQL generata:
-- SELECT * FROM video_games.game LIMIT 5;
SELECT id, genre_id, game_name 
FROM video_games.game 
LIMIT 5;

Spiegazione:


Anteprima risultati:
   id  genre_id                                game_name
0   1        11                              '98 Koshien
1   2         8               .hack//G.U. Vol.1//Rebirth
2   3         8             .hack//G.U. Vol.2//Reminisce
3   4         8  .hack//G.U. Vol.2//Reminisce (jp sales)
4   5         8            .hack//G.U. Vol.3//Redemption


In [11]:
print(llm_response)

```sql
-- SELECT * FROM video_games.game LIMIT 5;
SELECT id, genre_id, game_name 
FROM video_games.game 
LIMIT 5;
```

Explanation: This query selects the first 5 records from the `game` table in the `video_games` schema. The selected columns are `id`, `genre_id`, and `game_name`.
